In [235]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from multiprocessing import Process, Queue
import os
import numpy as np
import seaborn as sns
import tensorflow as tf
import pandas as pd
from scipy.misc import imsave
import scipy.stats
import random
from sklearn.metrics import accuracy_score

from problog import get_evaluatable
from problog.program import PrologString, SimpleProgram
from problog.core import ProbLog
from problog.logic import Constant,Var,Term,AnnotatedDisjunction
from problog.learning import lfi

%matplotlib notebook

In [232]:
st1 = """
st1 += "t(0.5)::gender_male;\n0.5::gender_female.\n"
st1 += "t(0.25)::age_1;\n0.25::age_2;\n0.25::age_3;\n0.25::age_4.\n"
"""

# st1 += "0.5::alcohol.\n"
# st1 += "0.5::smoke.\n"
# st1 += "0.5::blood_pressure.\n"
# st1 += "0.5::ldl.\n"
# st1 += "0.5::bmi.\n"
# st1 += "0.5::heart_d.\n"
# st1 += "0.5::glucose.\n"


In [233]:
st2 = ""
st2 += "t(0.5)::blood_pressure:-gender_male.\n"
st2 += "t(0.5)::blood_pressure:-gender_female.\n"

# st2 += "0.25::blood_pressure:-age_1.\n"
# st2 += "0.25::blood_pressure:-age_2.\n"
# st2 += "0.25::blood_pressure:-age_3.\n"
# st2 += "0.25::blood_pressure:-age_4.\n"

# st2 += "t(-)::blood_pressure:-alcohol.\n"
# st2 += "t(-)::blood_pressure:-smoke.\n"

In [283]:
model = """
t(0.5)::gender_male;t(0.5)::gender_female.
t(0.25)::age_1;t(0.25)::age_2;t(0.25)::age_3;t(0.25)::age_4.
t(0.5)::smoke_t;t(0.5)::smoke_f.
t(0.5)::alcohol_t;t(0.5)::alcohol_f.

t(0.33)::bmi_1;t(0.33)::bmi_2;t(0.33)::bmi_3.
t(0.5)::ldl_t;t(0.5)::ldl_f.
t(0.33)::blood_pressure_1;t(0.33)::blood_pressure_2;t(0.33)::blood_pressure_3.

t(_)::blood_pressure_1:-gender_male.
t(_)::blood_pressure_1:-gender_female.
t(_)::blood_pressure_1:-age_1.
t(_)::blood_pressure_1:-age_2.
t(_)::blood_pressure_1:-age_3.
t(_)::blood_pressure_1:-age_4.
t(_)::blood_pressure_1:-smoke_t.
t(_)::blood_pressure_1:-smoke_f.
t(_)::blood_pressure_1:-alcohol_t.
t(_)::blood_pressure_1:-alcohol_f.

t(_)::blood_pressure_2:-gender_male.
t(_)::blood_pressure_2:-gender_female.
t(_)::blood_pressure_2:-age_1.
t(_)::blood_pressure_2:-age_2.
t(_)::blood_pressure_2:-age_3.
t(_)::blood_pressure_2:-age_4.
t(_)::blood_pressure_2:-smoke_t.
t(_)::blood_pressure_2:-smoke_f.
t(_)::blood_pressure_2:-alcohol_t.
t(_)::blood_pressure_2:-alcohol_f.

t(_)::blood_pressure_3:-gender_male.
t(_)::blood_pressure_3:-gender_female.
t(_)::blood_pressure_3:-age_1.
t(_)::blood_pressure_3:-age_2.
t(_)::blood_pressure_3:-age_3.
t(_)::blood_pressure_3:-age_4.
t(_)::blood_pressure_3:-smoke_t.
t(_)::blood_pressure_3:-smoke_f.
t(_)::blood_pressure_3:-alcohol_t.
t(_)::blood_pressure_3:-alcohol_f.

t(_)::ldl_t:-gender_male.
t(_)::ldl_t:-gender_female.
t(_)::ldl_t:-age_1.
t(_)::ldl_t:-age_2.
t(_)::ldl_t:-age_3.
t(_)::ldl_t:-age_4.
t(_)::ldl_t:-smoke_t.
t(_)::ldl_t:-smoke_f.
t(_)::ldl_t:-alcohol_t.
t(_)::ldl_t:-alcohol_f.

t(_)::ldl_f:-gender_male.
t(_)::ldl_f:-gender_female.
t(_)::ldl_f:-age_1.
t(_)::ldl_f:-age_2.
t(_)::ldl_f:-age_3.
t(_)::ldl_f:-age_4.
t(_)::ldl_f:-smoke_t.
t(_)::ldl_f:-smoke_f.
t(_)::ldl_f:-alcohol_t.
t(_)::ldl_f:-alcohol_f.

t(_)::bmi_1:-gender_male.
t(_)::bmi_1:-gender_female.
t(_)::bmi_1:-age_1.
t(_)::bmi_1:-age_2.
t(_)::bmi_1:-age_3.
t(_)::bmi_1:-age_4.
t(_)::bmi_1:-smoke_t.
t(_)::bmi_1:-smoke_f.
t(_)::bmi_1:-alcohol_t.
t(_)::bmi_1:-alcohol_f.

t(_)::bmi_2:-gender_male.
t(_)::bmi_2:-gender_female.
t(_)::bmi_2:-age_1.
t(_)::bmi_2:-age_2.
t(_)::bmi_2:-age_3.
t(_)::bmi_2:-age_4.
t(_)::bmi_2:-smoke_t.
t(_)::bmi_2:-smoke_f.
t(_)::bmi_2:-alcohol_t.
t(_)::bmi_2:-alcohol_f.

t(_)::bmi_3:-gender_male.
t(_)::bmi_3:-gender_female.
t(_)::bmi_3:-age_1.
t(_)::bmi_3:-age_2.
t(_)::bmi_3:-age_3.
t(_)::bmi_3:-age_4.
t(_)::bmi_3:-smoke_t.
t(_)::bmi_3:-smoke_f.
t(_)::bmi_3:-alcohol_t.
t(_)::bmi_3:-alcohol_f.

t(_)::heart_d_t:-blood_pressure_1.
t(_)::heart_d_t:-blood_pressure_2.
t(_)::heart_d_t:-blood_pressure_3.
t(_)::heart_d_t:-ldl_t.
t(_)::heart_d_t:-ldl_f.
t(_)::heart_d_t:-bmi_1.
t(_)::heart_d_t:-bmi_2.
t(_)::heart_d_t:-bmi_3.

t(_)::heart_d_f:-blood_pressure_1.
t(_)::heart_d_f:-blood_pressure_2.
t(_)::heart_d_f:-blood_pressure_3.
t(_)::heart_d_f:-ldl_t.
t(_)::heart_d_f:-ldl_f.
t(_)::heart_d_f:-bmi_1.
t(_)::heart_d_f:-bmi_2.
t(_)::heart_d_f:-bmi_3.

t(_)::glucose_t:-blood_pressure_1.
t(_)::glucose_t:-blood_pressure_2.
t(_)::glucose_t:-blood_pressure_3.
t(_)::glucose_t:-ldl_t.
t(_)::glucose_t:-ldl_f.
t(_)::glucose_t:-bmi_1.
t(_)::glucose_t:-bmi_2.
t(_)::glucose_t:-bmi_3.

t(_)::glucose_f:-blood_pressure_1.
t(_)::glucose_f:-blood_pressure_2.
t(_)::glucose_f:-blood_pressure_3.
t(_)::glucose_f:-ldl_t.
t(_)::glucose_f:-ldl_f.
t(_)::glucose_f:-bmi_1.
t(_)::glucose_f:-bmi_2.
t(_)::glucose_f:-bmi_3.
"""

In [279]:
gender_male = Term('gender_male')
gender_female = Term('gender_female')

age_1 = Term('age_1')
age_2 = Term('age_2')
age_3 = Term('age_3')
age_4 = Term('age_4')

smoke_t = Term('smoke_t')
smoke_f = Term('smoke_f')

alcohol_t = Term('alcohol_t')
alcohol_f = Term('alcohol_f')

bmi_1 = Term('bmi_1')
bmi_2 = Term('bmi_2')
bmi_3 = Term('bmi_3')

ldl_t = Term('ldl_t')
ldl_f = Term('ldl_f')

blood_pressure_1 = Term('blood_pressure_1')
blood_pressure_2 = Term('blood_pressure_2')
blood_pressure_3 = Term('blood_pressure_3')

heart_d_t = Term('heart_d_t')
heart_d_f = Term('heart_d_f')

glucose_t = Term('glucose_t')
glucose_f = Term('glucose_f')

In [280]:
examples = [
    [(gender_female, True), (age_2, True), (blood_pressure_1, True)],
    [(gender_female, False), (age_2, False), (blood_pressure_2, False)]
]

In [281]:
score, weights, atoms, iteration, lfi_problem = lfi.run_lfi(PrologString(model), examples)

In [282]:
print (lfi_problem.get_model())

0.5::gender_male; 0.5::gender_female.
0.499999067783612::age_1; 0.5::age_2; 0.0::age_3; 0.0::age_4.
0.5::smoke_t; 0.5::smoke_f.
0.33::bmi_1; 0.33::bmi_2; 0.33::bmi_3.
0.5::ldl_t; 0.5::ldl_f.
0.656941554956785::blood_pressure_1; 0.0::blood_pressure_2; 0.33::blood_pressure_3.
0.172910968390091::blood_pressure_1 :- gender_male.
0.987187396620817::blood_pressure_1 :- gender_female.
0.595750561368152::blood_pressure_1 :- age_1.
0.999994891374277::blood_pressure_1 :- age_2.
0.456964683865635::blood_pressure_1 :- age_3.
0.807913642363929::blood_pressure_1 :- age_4.
0.0::blood_pressure_2 :- gender_male.
0.970214488096077::blood_pressure_2 :- gender_female.
0.0::blood_pressure_2 :- age_1.
0.576823128455931::blood_pressure_2 :- age_2.
0.509144520498797::blood_pressure_2 :- age_3.
0.018732259393713::blood_pressure_2 :- age_4.
0.687624876479627::blood_pressure_3 :- gender_male.
0.385101443051148::blood_pressure_3 :- gender_female.
0.309720241081935::blood_pressure_3 :- age_1.
0.428910342506444::bl